In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
mol_str = """
0 1
O
H 1 1.0
H 1 1.0 2 104.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)

cavity_options = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.1]),
    'ci_level' : 'cis',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'davidson_roots' : 1,
    'nact_orbs' : 0,
    'nact_els' : 0
}

psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [3]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 0

print(omega_val)

# lambda vector
lambda_vector = np.array([0., 0., 0.01])
#lambda_vector = np.array([0.5, 0.5, 0.5])

n_act_orb=5
#number of active electrons
n_act_el=4
# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)


0

Start SCF iterations:

Canonical RHF One-electron energy = -121.6753455400171902
CQED-RHF One-electron energy      = -121.6446277554181279
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0006736422568380
SCF Iteration   1: Energy = -74.9465252640783604   dE = -7.49465E+01   dRMS = 6.02250E-04
SCF Iteration   2: Energy = -74.9465445632316403   dE = -1.92992E-05   dRMS = 2.01092E-04
SCF Iteration   3: Energy = -74.9465467044282008   dE = -2.14120E-06   dRMS = 7.67758E-05
SCF Iteration   4: Energy = -74.9465470766861586   dE = -3.72258E-07   dRMS = 3.34663E-05
SCF Iteration   5: Energy = -74.9465471483387091   dE = -7.16526E-08   dRMS = 1.48018E-05
SCF Iteration   6: Energy = -74.9465471625286881   dE = -1.41900E-08   dRMS = 6.60930E-06
SCF Iteration   7: Energy = -74.9465471653629578   dE = -2.83427E-09   dRMS = 2.95662E-06
SCF Iteration   8: Energy = -74.9465471659306246   dE = -5.67667E-10   dRMS = 1.32357E-06
SCF Iteration   9: Energy = 

In [ ]:
print(H2_PF.H_1e_ao)


In [ ]:
_H_spin = np.einsum("uj,vi,uv", H2_PF.Ca, H2_PF.Ca, H2_PF.H_1e_ao)
print(_H_spin)

In [ ]:
_H_spin = np.repeat(_H_spin, 2, axis=0)
print(_H_spin)

In [ ]:
_H_spin = np.repeat(_H_spin, 2, axis=1)
print(_H_spin)

In [ ]:
spin_ind = np.arange(_H_spin.shape[0], dtype=int) % 2
print(spin_ind)

In [ ]:
Hspin = _H_spin * (spin_ind.reshape(-1, 1) == spin_ind)
print(spin_ind.reshape(-1, 1) == spin_ind)
print(Hspin)

In [ ]:
        if self.ignore_coupling == False:
            self.H_1e_ao += self.q_PF_ao + self.d_PF_ao
        # build H_spin
        # spatial part of 1-e integrals
        _H_spin = np.einsum("uj,vi,uv", self.Ca, self.Ca, self.H_1e_ao)
        _H_spin = np.repeat(_H_spin, 2, axis=0)
        _H_spin = np.repeat(_H_spin, 2, axis=0)
        # spin part of 1-e integrals
        spin_ind = np.arange(_H_spin.shape[0], dtype=int) % 2
        # product of spatial and spin parts
        self.Hspin = _H_spin * (spin_ind.reshape(-1, 1) == spin_ind)

In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

# davidson eigenvalues and eigenvectors are stored in attributes
# H2_PF.cis_e and H2_PF.cis_c for now!


In [ ]:
print("checking to see if the Davidson Roots Agree!")
print(e_fci[:14])
print(H2_PF.cis_e[:14] )
print(np.allclose(e_fci[:14] , H2_PF.cis_e[:14]))


In [ ]:

H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




In [ ]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)